In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

In [ ]:
!pip3 install hanziconv

In [ ]:
def sumMinDistance(a:torch.tensor,b:torch.tensor):
    return float(torch.sum(torch.tensor(minDistance(a,b))))

def minDistance(a:torch.tensor,b:torch.tensor):
    return [float(torch.topk(i,1,largest=False).values) for i in torch.cdist(a, b, p=1)]
a = torch.tensor([[2.,2.],[1.,1.]])
b = torch.tensor([[1.,1.],[9.,9.]])
a = torch.rand(128,128)
b = torch.rand(128,128)

minDistance(a,b)
sumMinDistance(a,b)

In [ ]:
import torch
torch.manual_seed(123) 

#
x = torch.randn(4, 3)  # 输入的维度是（128，20）
print(f"x:{x}")
print(f"x.shape:{x.shape}")


m = torch.nn.Linear(3, 2)  # 20,30是指维度
print('m.weight.shape:\n ', m.weight.shape)
print('m:\n ', m)
print('m.bias.shape:\n', m.bias.shape)

#
output = m(x)
print('output.shape:\n', output.shape)
print('output.shape:\n', output)



In [13]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        #print(f"log_probs: {log_probs}")
        return(log_probs)
    
#
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


def get_word_vec(word,word_to_ix,device="cpu"):
    context_ids = make_context_vector(word, word_to_ix).to(device)
    assert context_ids.is_cuda is True # returns a boolean
    return model.embeddings(context_ids)

In [ ]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
print(f"vocab: {vocab}")

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(f"data:{data[:5]}")

#

print(f"make_context_vector: {make_context_vector(data[0][0], word_to_ix)}")


In [ ]:
#
device = torch.device('cuda:0')
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), embedding_dim=10, context_size=CONTEXT_SIZE*2)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
get_word_vec([data[0][0][0]],word_to_ix,device)

In [ ]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        context_ids = make_context_vector(context, word_to_ix)
        context_ids = context_ids.to(device)
        model.zero_grad()
        log_probs = model(context_ids)
        label = torch.tensor([word_to_ix[target]], dtype=torch.long)
        label = label.to(device)
        loss = loss_function(log_probs, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

In [ ]:
get_word_vec([data[0][0][0]],word_to_ix,device)


## 中文

In [1]:
!pip3 install jieba


In [8]:
import jieba

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
text = "華航諾富特飯店群聚案新增首例外包商水電工案1145確診，中央流行疫情指揮中心持續追查水電工感染源，衛福部長陳時中與疾管署長周志浩今天上午一同到台大醫院參加「清潔雙手，攜手抗疫」記者會，會前陳時中受訪透露，案1145可能曾到過諾富特飯店一館B1用餐，不排除任何感染可能。指揮中心昨天公佈的案1145，是在諾富特飯店的一館3、5、6樓層（整修樓層）工作，由於和飯店人員、旅客都沒有接觸，因此感染源不明；不過陳時中今天表示，在施工的地方大部分都是工作人員，但是可能有在B1的地方共餐。媒體追問，是否表示案1145有去過B1，是不是可能被案1120或是餐飲部的員工感染？陳時中回應，各種可能性都有，但還需要相關的釐清。"
# raw_text = [t for t in text]
raw_text = list(jieba.cut(text))

vocab = set(raw_text)
vocab_size = len(vocab)
print(f"{vocab_size},vocab: {vocab}")

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(f"data:{data[:5]}")

#

print(f"make_context_vector: {make_context_vector(data[0][0], word_to_ix)}")

116,vocab: {'表示', '但還', '追查', '飯店', '各種', '於', '工案', '是不是', '由', '醫院', '、', '相關', '，', '（', '大部分', '記者', '的', '參加', '）', '時', '到', '不', '共餐', '？', '媒體', '新增', '5', '公', '施工', '源', '「', '今天', '釐', '都', '是否', '會前', '或是', '台大', '餐飲部', '水電', '佈', '沒', '去過', '樓層', '周志浩', '包商', '6', '但是', '昨天', '衛福部長', '任何', '確診', '曾到', '持續', '清', '在', '中', '一同', '」', '一館', '受訪', '和', '中心', '會', '1120', '1145', '攜手', '可能性', '清潔', '抗疫', '例外', '人員', '特飯店', '排除', '用餐', '接觸', '特飯', '被案', '是', '旅客', '回應', '華航諾富', '群聚', '透露', '今天上午', 'B1', '地方', '感染', '水電工', '陳', '追問', '不明', '店', '疫情', '過諾富', '因此', '。', '工作', '3', '不過', '可能', '首', '與', '中央', '整修', '指揮', '疾管署', '雙手', '諾富', '；', '長', '需要', '流行', '案', '員工', '有'}
data:[(['華航諾富', '特飯', '群聚', '案'], '店'), (['特飯', '店', '案', '新增'], '群聚'), (['店', '群聚', '新增', '首'], '案'), (['群聚', '案', '首', '例外'], '新增'), (['案', '新增', '例外', '包商'], '首')]
make_context_vector: tensor([ 81,  76,  82, 113])


In [11]:
#
embedding_dim_num = 30
device = torch.device('cuda:0')
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), embedding_dim=embedding_dim_num, context_size=CONTEXT_SIZE*2)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [12]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        context_ids = make_context_vector(context, word_to_ix)
        context_ids = context_ids.to(device)
        model.zero_grad()
        log_probs = model(context_ids)
        label = torch.tensor([word_to_ix[target]], dtype=torch.long)
        label = label.to(device)
        loss = loss_function(log_probs, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

log_probs: tensor([[-4.8169, -5.0936, -4.9221, -4.7769, -4.8293, -4.5590, -4.7302, -4.5362,
         -4.5257, -5.0402, -5.1396, -4.9084, -4.6884, -4.6830, -4.4518, -4.8137,
         -5.0262, -4.4787, -4.5926, -4.8034, -4.4789, -4.7796, -4.4844, -4.5561,
         -4.8852, -5.0358, -4.7488, -4.7720, -4.6596, -4.7671, -4.5819, -4.7792,
         -4.4002, -4.3510, -4.6563, -5.0557, -4.7620, -4.9075, -4.7095, -4.8442,
         -4.6207, -4.8074, -5.1036, -4.8243, -5.0813, -4.6898, -5.0201, -4.5324,
         -4.6590, -4.4081, -4.5290, -4.5346, -4.6163, -4.4335, -4.7968, -4.8566,
         -4.6328, -5.0184, -4.9603, -4.5755, -5.1219, -4.7491, -4.7266, -5.0321,
         -4.6469, -5.0455, -4.6758, -4.4241, -5.3311, -4.9489, -4.9791, -4.9108,
         -4.9074, -5.0400, -4.6641, -4.8194, -4.2999, -4.7140, -4.7156, -4.9601,
         -5.0429, -5.0603, -4.5884, -4.7403, -4.7594, -4.9260, -4.7904, -4.7108,
         -4.6050, -4.4640, -4.8601, -4.7667, -5.3106, -4.3609, -5.1122, -5.2175,
         -4.8192,

log_probs: tensor([[-5.2475, -5.0563, -4.7502, -5.1180, -5.1318, -4.9898, -4.9803, -5.4015,
         -5.1236, -4.8652, -5.2371, -5.0912, -4.5890, -5.1140, -5.3935, -5.0587,
         -4.9098, -5.3010, -4.9688, -4.8455, -4.5667, -4.9157, -4.9602, -5.5322,
         -5.1117, -5.0027, -4.7420, -4.4683, -4.9927, -4.4417, -4.3558, -5.0802,
         -4.9383, -4.9603, -5.0504, -4.4092, -5.1881, -4.4045, -4.9835, -4.7604,
         -4.6629, -4.8651, -5.0121, -5.3421, -4.8949, -4.6663, -5.3984, -4.9616,
         -4.0199, -4.3894, -5.1053, -4.3220, -4.9943, -4.7768, -5.2065, -4.9338,
         -4.6721, -5.1042, -4.1917, -4.6627, -4.4512, -5.5939, -3.8607, -4.0913,
         -4.9253, -3.0975, -4.7724, -4.8900, -4.9158, -4.8759, -4.2771, -5.4487,
         -4.4045, -4.2143, -4.4812, -5.3733, -4.8145, -5.1798, -5.0014, -4.9797,
         -4.9318, -5.1782, -5.1354, -4.9720, -4.7727, -5.0538, -4.8933, -3.9272,
         -4.4442, -4.8733, -5.1883, -5.0408, -4.9276, -5.0225, -4.4346, -5.1844,
         -4.8243,

log_probs: tensor([[-4.1712, -5.8244, -5.9662, -5.6096, -6.5673, -5.1081, -5.9466, -5.5866,
         -5.8302, -5.6107, -3.8617, -5.4762, -2.3587, -5.2549, -5.0546, -6.1485,
         -1.7040, -5.8156, -5.5398, -3.6693, -5.8079, -6.3599, -5.1007, -5.4609,
         -5.0224, -3.3815, -4.9778, -6.3429, -5.2242, -5.9025, -6.0502, -6.5616,
         -5.6196, -3.3089, -5.6141, -6.0528, -5.5445, -6.4798, -5.4623, -6.0171,
         -5.8583, -5.5879, -5.7345, -4.8998, -5.7636, -3.5850, -5.2857, -5.7612,
         -5.5220, -6.1335, -5.5764, -5.8397, -5.0969, -5.5515, -6.0355, -4.9537,
         -4.2725, -5.8838, -4.8000, -5.5994, -6.2908, -5.7495, -6.0457, -5.7462,
         -5.3323, -6.2868, -6.1743, -5.9240, -6.1998, -5.8758, -4.9307, -4.2850,
         -5.0854, -5.0189, -5.5963, -5.7701, -5.2217, -5.3350, -4.9033, -5.5228,
         -5.3585, -6.5907, -4.0334, -5.9333, -5.6221, -5.0341, -3.9056, -5.3689,
         -5.7170, -3.6147, -5.8540, -5.5140, -4.7519, -6.0596, -5.9975, -5.2560,
         -5.1322,

log_probs: tensor([[-5.4429, -5.6794, -5.2986, -4.5271, -5.5082, -4.9176, -5.3227, -5.8155,
         -4.7506, -5.0004, -3.6367, -6.3300, -8.7508, -6.0095, -5.8952, -5.3504,
         -4.8499, -5.7360, -4.5306, -6.5325, -4.0584, -6.1226, -6.0422, -5.3749,
         -6.0550, -6.1118, -4.3372, -5.8300, -5.5862, -6.8488, -5.1452, -5.7358,
         -6.0302, -2.6928, -5.9222, -5.2128, -6.0291, -5.3853, -6.0812, -5.4451,
         -5.2563, -2.5717, -6.1878, -3.8538, -5.1041, -5.4348, -5.3337, -5.2748,
         -3.1822, -4.9957, -4.1437, -5.4102, -6.6343, -6.2484, -6.7101, -4.9192,
         -5.8134, -5.3404, -5.4572, -2.8890, -4.1027, -5.7868, -3.5971, -3.8936,
         -5.8159, -4.0504, -5.0679, -5.9116, -5.3591, -5.4066, -5.5048, -4.0058,
         -5.4753, -4.1333, -4.5845, -5.6787, -6.3567, -5.5322, -4.5811, -4.8004,
         -6.0836, -6.7370, -6.5730, -5.5251, -6.2684, -5.7407, -5.6002, -3.9618,
         -5.1060, -5.0267, -5.4924, -5.9745, -4.8881, -5.9915, -4.2453, -5.8258,
         -4.9703,

log_probs: tensor([[-5.5542, -5.6823, -6.3768, -6.9193, -7.4888, -6.3843, -6.8891, -5.7045,
         -7.2761, -5.9404, -8.4270, -6.0748, -0.4599, -8.1007, -8.2832, -7.3822,
         -5.2928, -6.0802, -6.2696, -8.2498, -7.5745, -7.7618, -4.6956, -6.5521,
         -6.3721, -6.8850, -7.7744, -7.4657, -8.2506, -6.6395, -8.1324, -6.9259,
         -6.9459, -7.4574, -6.8264, -7.7646, -5.5281, -6.4378, -5.6274, -5.3392,
         -7.3735, -8.6882, -6.7387, -7.8076, -6.9181, -7.0599, -8.3847, -5.9620,
         -8.1861, -4.9262, -6.7264, -5.8078, -7.2634, -5.7670, -8.1766, -6.2035,
         -8.8759, -7.6498, -8.3899, -7.9020, -7.5293, -7.6804, -6.7258, -7.0675,
         -6.7101, -8.7480, -7.3740, -7.0452, -6.1525, -7.7444, -6.1176, -7.1265,
         -6.3394, -8.3504, -7.1824, -6.0367, -4.5055, -6.7722, -6.9519, -6.5902,
         -5.8960, -8.0984, -5.8772, -8.0631, -7.0381, -3.1347, -6.9975, -6.8490,
         -7.1550, -6.8345, -6.6429, -6.6117, -7.7955, -7.9226, -8.3753, -8.1155,
         -7.4686,

...

log_probs: tensor([[-9.0649e+01, -1.0602e+02, -1.0740e+02, -1.0044e+02, -1.0743e+02,
         -4.9373e+01, -9.2274e+01, -1.0756e+02, -1.0772e+02, -1.1351e+02,
         -1.3590e+02, -8.7508e+01, -2.5783e+02, -9.9964e+01, -1.5345e+01,
         -1.1096e+02, -1.7413e+02, -9.9241e+01, -1.4076e+02, -1.3326e+02,
         -9.2211e+01, -1.1910e+02, -9.6394e+01, -1.0688e+02, -1.0477e+02,
         -9.7271e+01, -1.0283e+02, -1.1022e+02, -1.0231e+02, -1.1943e+02,
         -1.0532e+02, -1.0001e+02, -1.3244e+02, -1.3735e+02, -8.6635e+01,
         -1.1446e+02, -1.0194e+02, -9.4792e+01, -1.2887e+02, -1.1949e+02,
         -8.7660e+01, -7.3935e+01, -1.2842e+02, -2.6698e+02, -1.2741e+02,
         -1.1596e+02, -1.1388e+02, -9.4203e+01, -8.4305e+01, -9.1922e+01,
         -1.0345e+02, -6.5052e+01, -1.1184e+02, -1.5806e+02, -1.1888e+02,
         -1.0697e+02, -1.6523e+02, -1.2711e+02, -1.1370e+02, -1.1829e+02,
         -1.3617e+02, -9.6541e+01, -6.9914e+01, -9.3306e+01, -1.1066e+02,
         -2.5959e+01, -9.43

log_probs: tensor([[ -0.0978, -29.8981, -29.9659, -22.5477, -35.3543, -32.1474, -25.8512,
         -27.6885, -23.6863, -34.4304, -29.1386, -32.5014, -20.0766, -18.2416,
         -30.8387, -30.9490, -44.1034, -33.9395, -41.3280, -29.7317, -27.3073,
         -34.7797, -27.9669, -38.5225, -26.4583, -25.0663, -33.2890, -24.5942,
         -28.5561, -31.0535, -24.8748, -21.1339, -34.6325, -51.6923, -23.5953,
         -27.6334, -28.5577, -19.4748, -50.0051, -31.3381, -29.3360, -30.5777,
         -26.0211, -43.5131, -53.5123, -37.8834, -30.1060, -31.4055, -32.1722,
         -19.6726, -26.5805, -17.7893, -26.7395, -35.3491, -30.2804, -34.9039,
         -47.2998, -32.2616,  -2.4010, -35.4670, -28.1206, -16.4154, -33.3899,
          -5.9685, -33.4315, -36.7341, -18.0707, -31.0361, -23.8922, -32.2468,
         -31.6070, -42.6023, -32.6390, -33.2361, -26.8329, -22.2352, -25.3780,
         -29.1002, -19.6456, -25.7665, -17.9380, -32.4939, -25.9493, -11.0100,
         -29.8543, -38.3822, -24.9535, -3

log_probs: tensor([[-52.0280, -52.0511, -52.0242, -50.6448, -57.9146, -36.0882, -50.9749,
         -44.9815, -46.2585, -60.8870, -66.8814, -52.7061, -64.3715, -64.8829,
         -56.9782, -42.3047,   0.0000, -59.5604, -44.8904, -51.3711, -36.3208,
         -55.5566, -49.4120, -53.4256, -37.0415, -57.8148, -42.4855, -51.5222,
         -49.6059, -82.5202, -53.4626, -42.1279, -61.8149, -59.8337, -55.5182,
         -56.3585, -50.8298, -57.7905, -75.5147, -53.6838, -51.4156, -45.8802,
         -47.1208, -69.9763, -71.2811, -46.7952, -68.8995, -54.6539, -56.8563,
         -48.9935, -43.3822, -39.7896, -58.5568, -56.6754, -54.7701, -56.7939,
         -77.7284, -47.0911, -42.6608, -58.7417, -46.4579, -52.1922, -57.2284,
         -29.9322, -52.1906, -65.2116, -57.4897, -53.2948, -54.2066, -57.1639,
         -58.5990, -61.7849, -47.7390, -64.6565, -48.4612, -47.8151, -54.4196,
         -58.0520, -64.6019, -41.9861, -54.9384, -54.6801, -52.5631, -48.3537,
         -54.4170, -45.6495, -53.0110, -5

log_probs: tensor([[-14.8806, -12.8155, -11.4907, -12.8033, -12.0069, -18.6859, -15.5658,
          -9.6411,  -7.3005, -26.7693, -12.1194, -15.1409, -13.9573, -21.3440,
         -11.5448, -10.2193, -17.6152,  -8.5488, -15.3216, -15.4740, -16.7318,
          -8.0781, -13.1806, -15.7879, -12.6210, -15.6666, -14.9214, -17.2763,
          -6.6809, -17.8417,  -7.1758, -17.0792, -11.0810, -19.3197,  -5.9874,
          -9.9393, -13.7142, -17.6250, -22.5454, -11.9282,  -2.6244, -14.4150,
         -11.3155, -25.0864, -15.4068, -21.4713,  -9.7441, -11.7428, -18.1428,
         -10.3332, -12.2495, -15.7597,  -2.2218, -15.9393, -14.9217, -13.8825,
         -12.3476,  -8.5696, -11.8249, -14.0001,  -9.6226, -21.6086, -13.9081,
         -16.4966, -12.5799, -23.5043,  -9.4184, -11.6013, -10.5480, -15.0739,
         -26.3724, -11.4738, -16.8153, -15.9567,  -1.1704,  -7.0182, -18.2243,
         -15.4376,  -6.7849, -15.0523,  -7.7486, -14.8628,  -9.0986,  -4.7147,
         -17.8434, -22.7905, -11.6690, -2

log_probs: tensor([[ -82.0044,  -70.0459,  -68.0128,  -71.0485,  -73.8792,  -65.5256,
          -64.6038,  -66.5014,  -61.9610, -109.0576,  -69.8773,  -17.4528,
          -84.0395,  -82.2006,  -63.9613,  -66.9495,  -91.7153,  -47.7519,
          -85.2913,  -87.1439,  -62.4065,  -67.2322,  -59.1389,  -80.0111,
          -77.9922,  -46.3931,  -72.1339,  -65.0343,  -62.8863,  -76.0160,
          -58.5443,  -75.1529,  -70.2286,  -90.9459,  -81.9681,  -69.7939,
          -52.1289,  -48.4044,  -84.3876,  -79.2994,  -65.5338,  -68.4622,
          -76.8809, -113.5593,  -37.7526,  -56.5283,  -89.9106,  -74.9384,
          -50.2763,  -75.9489,  -71.7811,  -61.2269,  -73.1196,  -94.7597,
          -77.4704,  -72.0374,  -88.3284,  -74.0905,  -67.8671,  -89.3958,
          -90.4927,  -93.5203,  -91.8741,  -56.0155,  -76.3530,  -69.0167,
          -60.9726,  -73.1730,  -63.4555,  -72.4583,  -72.7507,    0.0000,
          -77.0705,  -60.8587,  -89.5592,  -69.7792,  -61.0014,  -79.3143,
          -67.

In [13]:
print(f"text:{[data[0][0][0]]}, vec: {get_word_vec([data[0][0][0]],word_to_ix,device)}")

text:['華航諾富'], vec: tensor([[ 2.0484,  0.6284,  0.9729, -2.7307,  1.6630,  0.4887,  1.2890, -0.4918,
          0.3060,  1.3692,  0.4008, -1.1012,  0.2481, -0.3195,  1.6232,  0.2476,
         -0.1606,  0.0896,  1.9680,  0.3584,  0.7102, -0.6327, -1.5741,  1.2044,
         -0.1513,  1.0809, -0.4869,  0.2361, -0.1189,  1.3261]],
       device='cuda:0', grad_fn=<EmbeddingBackward>)


In [14]:
a = get_word_vec([data[0][0][0]],word_to_ix,device)
b = get_word_vec([data[0][0][1]],word_to_ix,device)
torch.cdist(a, b, p=embedding_dim_num)

tensor([[3.7538]], device='cuda:0', grad_fn=<CdistBackward>)

In [15]:
import torch
PATH_MODEL_SAVE = "./save_word2vec.pt"

# Save:
torch.save(model.state_dict(), PATH_MODEL_SAVE)

In [10]:
import torch
# Load:
CONTEXT_SIZE = 2
VOCAB_SIZE=116
embedding_dim_num =30
PATH_MODEL_SAVE = "./save_word2vec.pt"
device = torch.device('cuda:0')


model = CBOW(VOCAB_SIZE, embedding_dim=embedding_dim_num, context_size=CONTEXT_SIZE*2)
model.load_state_dict(torch.load(PATH_MODEL_SAVE))
model.to(device)
get_word_vec(["華航諾富"],word_to_ix,device)


tensor([[ 0.5101, -2.9264, -0.9242, -1.1863,  0.0423,  1.9820,  0.0124,  1.2620,
         -0.8510, -1.9742,  0.5986, -0.1720,  1.1483,  1.4214, -0.1714,  0.8528,
         -1.4997,  0.9899,  0.5618, -0.5556, -0.7017, -1.3265,  0.4407,  1.9369,
         -2.2337, -1.5637, -0.6424,  1.4863, -0.2650,  0.6148]],
       device='cuda:0', grad_fn=<EmbeddingBackward>)

# 訓練中文字向量

In [6]:
import json
import jieba

article_cnt = 10000
with open("./train_wiki.json") as f:
    data = json.load(f)
print(f"article_cnt: {article_cnt}/{len(data)}")
data = data[:article_cnt]

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
text = "。。。。".join(data)
# raw_text = [t for t in text]
raw_text = list(jieba.cut(text))

vocab = set(raw_text)
vocab_size = len(vocab)
print(f"vocab_size: {vocab_size}")

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(f"data:{data[:15]}")

article_cnt: 10000/1043224
vocab_size: 212622
data:[(['劉群', '劉群', '）', '，'], '（'), (['劉群', '（', '，', '重慶市'], '）'), (['（', '）', '重慶市', '人'], '，'), (['）', '，', '人', '，'], '重慶市'), (['，', '重慶市', '，', '漢族'], '人'), (['重慶市', '人', '漢族', '，'], '，'), (['人', '，', '，', '中華'], '漢族'), (['，', '漢族', '中華', '人民'], '，'), (['漢族', '，', '人民', '共和'], '中華'), (['，', '中華', '共和', '國'], '人民'), (['中華', '人民', '國', '政治'], '共和'), (['人民', '共和', '政治', '人物'], '國'), (['共和', '國', '人物', '、'], '政治'), (['國', '政治', '、', '第十二'], '人物'), (['政治', '人物', '第十二', '屆'], '、')]


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import datetime



#
epoch_size = 5
embedding_dim_num = 30
device = torch.device('cuda:0')
# losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), embedding_dim=embedding_dim_num, context_size=CONTEXT_SIZE*2)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)
print(f"data size:{len(data)}")




data size:2405684


In [ ]:
#
print(f"training...")
for epoch in range(epoch_size):
    total_loss = torch.Tensor([0])
    idx = 0
    start_datetime = datetime.datetime.now()
    for context, target in data:

        context_ids = make_context_vector(context, word_to_ix)
        context_ids = context_ids.to(device)
        model.zero_grad()
        log_probs = model(context_ids)
        label = torch.tensor([word_to_ix[target]], dtype=torch.long)
        label = label.to(device)
        loss = loss_function(log_probs, label)
        if idx % 1000 == 0 and idx !=0:  
            print(f"{epoch}-loss-{idx}: {total_loss/1000}, time: {datetime.datetime.now()-start_datetime}")
            start_datetime = datetime.datetime.now()
            total_loss = torch.Tensor([0])

        if idx % 20000 ==0 and idx !=0:
            print(f"save model {idx}")
            torch.save(model.state_dict(),f"./save_wiki_word2vec-{idx}.pt")


        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        idx+=1
    print(f"{epoch}-total_loss: {total_loss}")
    #losses.append(total_loss)
# print(losses)

training...
0-loss-1000: tensor([8.5588]), time: 0:00:06.183615
0-loss-2000: tensor([8.6932]), time: 0:00:06.164135
0-loss-3000: tensor([8.9243]), time: 0:00:06.166802
0-loss-4000: tensor([8.6833]), time: 0:00:06.167031
0-loss-5000: tensor([8.8056]), time: 0:00:06.166583
0-loss-6000: tensor([8.9202]), time: 0:00:06.166080
0-loss-7000: tensor([8.9886]), time: 0:00:06.165489
0-loss-8000: tensor([9.0588]), time: 0:00:06.165496
0-loss-9000: tensor([8.9898]), time: 0:00:06.165713
0-loss-10000: tensor([8.9319]), time: 0:00:06.167174
0-loss-11000: tensor([8.7935]), time: 0:00:06.169790
0-loss-12000: tensor([8.9086]), time: 0:00:06.169300
0-loss-13000: tensor([8.8649]), time: 0:00:06.169390
0-loss-14000: tensor([9.3237]), time: 0:00:06.169511
0-loss-15000: tensor([8.7131]), time: 0:00:06.169328
0-loss-16000: tensor([9.1743]), time: 0:00:06.168648
0-loss-17000: tensor([8.8287]), time: 0:00:06.168669
0-loss-18000: tensor([9.0952]), time: 0:00:06.169591
0-loss-19000: tensor([8.8556]), time: 0:00:

In [33]:
import torch
# Load:
PATH_MODEL_SAVE = "./save_wiki_word2vec-60000.pt"
CONTEXT_SIZE = 2
VOCAB_SIZE=212622
embedding_dim_num =30
device = torch.device('cuda:0')


model = CBOW(VOCAB_SIZE, embedding_dim=embedding_dim_num, context_size=CONTEXT_SIZE*2)
model.load_state_dict(torch.load(PATH_MODEL_SAVE))
model.to(device)
get_word_vec(["劉群"],word_to_ix,device)


tensor([[ 1.7494e-01,  6.4115e-01, -8.1290e-01, -1.7062e+00,  1.7668e-01,
          1.0450e+00,  9.9193e-01, -4.1422e-01,  5.0230e-01,  1.9129e+00,
         -1.3418e+00,  2.4677e-01, -1.1381e+00,  3.2698e-01, -1.7989e+00,
          1.4358e-01,  2.6641e-01,  1.3021e+00,  1.2260e+00, -2.6347e+00,
         -6.3820e-01,  5.8675e-05, -4.3886e-01, -3.8941e-01, -8.6265e-01,
          7.7250e-01, -8.2313e-01,  6.3576e-01,  1.7220e+00,  6.6429e-01]],
       device='cuda:0', grad_fn=<EmbeddingBackward>)